In [ ]:
import pandas as pd
import numpy as np

In [1]:
import pandas as pd

# Define the file path
file_path = r"C:\Users\user\Desktop\iit\year2 sem1\sdgp\AgroEdge\server\ML_part\price_forcasting\vegetable_fruit_prices.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Display the first few rows
df.head()


,Date,Commodity,Market Region,Price per Unit (LKR/kg)
0,2020-02-13,Winged Bean,Ampara,376.43
1,2020-02-13,Winged Bean,Anuradhapura,210.15
2,2020-02-13,Winged Bean,Badulla,362.29
3,2020-02-13,Winged Bean,Batticaloa,169.92
4,2020-02-13,Winged Bean,Colombo,372.34


In [ ]:
df.info()

In [ ]:
# Count missing values in each column
print(df.isnull().sum())

In [ ]:
# Drop exact duplicate rows
df = df.drop_duplicates()

print("No of duplicates:", df.duplicated().sum())

In [ ]:
# convert the dates to date time format
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
import numpy as np

Q1 = df['Price per Unit (LKR/kg)'].quantile(0.25)
Q3 = df['Price per Unit (LKR/kg)'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = df[(df['Price per Unit (LKR/kg)'] < lower_bound) | (df['Price per Unit (LKR/kg)'] > upper_bound)]
print("Number of outliers:", len(outliers))

In [ ]:
df['Commodity'] = df['Commodity'].str.strip().str.lower()
df['Commodity'] = df['Commodity'].str.strip().str.lower()

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# General Price Distribution
plt.figure(figsize=(10, 5))
sns.histplot(df['Price per Unit (LKR/kg)'], bins=50, kde=True)
plt.title("Distribution of Prices")
plt.xlabel("Price per Unit (LKR/kg)")
plt.ylabel("Frequency")
plt.grid()
plt.show()


In [ ]:
# Boxplot for detecting outliers
plt.figure(figsize=(8, 5))
sns.boxplot(x=df['Price per Unit (LKR/kg)'])
plt.title("Boxplot of Prices (Outlier Detection)")
plt.xlabel("Price per Unit (LKR/kg)")
plt.grid()
plt.show()

In [ ]:
# Time Series Plot for Average Price Trend
df_avg_price = df.groupby("Date")["Price per Unit (LKR/kg)"].mean()

plt.figure(figsize=(12, 6))
plt.plot(df_avg_price.index, df_avg_price.values, marker="o", linestyle="-", label="Average Price")
plt.title("Time Series Trend of Average Prices")
plt.xlabel("Date")
plt.ylabel("Average Price")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Show unique crop and city counts
unique_crops = df['Commodity'].nunique()
unique_cities = df['Market Region'].nunique()

# Display results
unique_crops, unique_cities

In [ ]:
from sklearn.preprocessing import LabelEncoder
from IPython.display import display  
import pandas as pd


# Encode categorical variables
crop_encoder = LabelEncoder()
city_encoder = LabelEncoder()

df["Commodity_Encoded"] = crop_encoder.fit_transform(df["Commodity"])
df["Market_Region_Encoded"] = city_encoder.fit_transform(df["Market Region"])

# Drop original categorical columns
df_encoded = df.drop(columns=["Commodity", "Market Region"])

# Display first few rows of the updated dataset
display(df_encoded)  

In [ ]:
# Retrieve the mapping for Commodity
commodity_mapping = {index: value for index, value in enumerate(crop_encoder.classes_)}
print("\nCommodity Encoding Mapping:")
for key, val in commodity_mapping.items():
    print(f"{val} -> {key}")

# Retrieve the mapping for Market Region
market_region_mapping = {index: value for index, value in enumerate(city_encoder.classes_)}
print("\nMarket Region Encoding Mapping:")
for key, val in market_region_mapping.items():
    print(f"{val} -> {key}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Apply MinMaxScaler to price column
scaler = MinMaxScaler()
df["Price_Scaled"] = scaler.fit_transform(df[["Price per Unit (LKR/kg)"]])

# Drop unnecessary columns
df_scaled = df.drop(columns=["Price per Unit (LKR/kg)", "Commodity", "Market Region"])

# Display the transformed dataset in Jupyter Notebook
display(df_scaled)  # Use this in Jupyter Notebook

In [ ]:
from statsmodels.tsa.stattools import adfuller  # Import ADF test function

# Reduce dataset size for ADF test (sampling 5000 records)
sample_size = 5000
df_sample = df.sample(n=sample_size, random_state=42)

# Run ADF test on the sampled data
adf_test_result = adfuller(df_sample["Price_Scaled"])

# Extract the p-value
stationarity_p_value = adf_test_result[1]

# Determine stationarity
is_stationary = "Stationary" if stationarity_p_value < 0.05 else "Non-Stationary"

# Display results
print(f"ADF Test p-value: {stationarity_p_value}")
print(f"Dataset is: {is_stationary}")


In [ ]:
df_preprocessed = df_scaled

In [ ]:
df_preprocessed

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
try:
    print("Checking dataset...")
    print(df_preprocessed.head())  # Display first rows to confirm dataset is loaded
except NameError:
    print("Error: df_preprocessed is not defined. Load the dataset before running the model.")
    raise SystemExit

# Print unique values for debugging
print("Unique Commodities:", df_preprocessed["Commodity_Encoded"].unique())
print("Unique Regions:", df_preprocessed["Market_Region_Encoded"].unique())

In [ ]:
def prepare_lstm_data(series, n_lags):
    """Prepare time series data for LSTM."""
    X, y = [], []
    for i in range(len(series) - n_lags):
        X.append(series[i:i + n_lags])
        y.append(series[i + n_lags])
    return np.array(X), np.array(y)

In [ ]:
selected_commodity = 5  # Example: Change based on user selection (I added this just for cheking the input)
selected_region = 12     # Example: Change based on user selection (I added this just for cheking the input)

# Check if the selected commodity and region exist
if selected_commodity not in df_preprocessed["Commodity_Encoded"].unique():
    raise ValueError(f"Commodity {selected_commodity} not found in dataset.")
if selected_region not in df_preprocessed["Market_Region_Encoded"].unique():
    raise ValueError(f"Region {selected_region} not found in dataset.")

In [ ]:
df_filtered = df_preprocessed[
    (df_preprocessed["Commodity_Encoded"] == selected_commodity) & 
    (df_preprocessed["Market_Region_Encoded"] == selected_region)
]

# Ensure the dataset is sorted by date
df_filtered = df_filtered.sort_values("Date")

# Convert "Date" column to datetime format
df_filtered["Date"] = pd.to_datetime(df_filtered["Date"])

# Check if 'Price_Scaled' exists
if "Price_Scaled" not in df_filtered.columns:
    raise KeyError("Column 'Price_Scaled' not found in dataset.")

# Use 'Price_Scaled' as the target variable
prices = df_filtered["Price_Scaled"].values

# Define the number of past weeks to consider for forecasting
n_lags = 12  # Using past 12 weeks to predict the next 12 weeks

# Prepare input features and target labels
X, y = prepare_lstm_data(prices, n_lags)

# Check for empty arrays
if len(X) == 0 or len(y) == 0:
    raise ValueError("Not enough data points available for training. Try selecting a different commodity or region.")

# Reshape for LSTM (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Check shapes before training
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define Callbacks (Learning Rate Scheduler & Early Stopping)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Reduce LR if val_loss plateaus
    factor=0.5,  
    patience=5,  
    min_lr=1e-6  
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # Stop training if val_loss doesn't improve
    patience=10,  
    restore_best_weights=True  # Restore best weights when stopping
)

# Final Optimized Model
model = Sequential([
    Input(shape=(n_lags, 1)),  # Fix input shape warning
    LSTM(128, activation='relu', return_sequences=True),  # Increased LSTM units (from 64 → 128)
    Dropout(0.15),  
    LSTM(128, activation='relu'),  
    Dropout(0.15),
    Dense(1)  
])

#  Compile Model with Adjusted Learning Rate
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

#  Train Model with Callbacks
history = model.fit(
    X_train, y_train, 
    epochs=100,  # Let early stopping decide actual stopping point
    batch_size=32,  
    validation_data=(X_test, y_test),  
    verbose=1,  
    callbacks=[lr_scheduler, early_stopping]  # Include both optimizations
)
